In [ ]:
!pip install torchaudio

     |████████████████████████████████| 1.9MB 7.7MB/s 


In [ ]:
import torch
import torchaudio
from torchaudio import functional as F
from torchaudio import transforms as T

In [ ]:
torchaudio.set_audio_backend("sox_io")

In [ ]:
hop_length = 882
n_fft = hop_length * 2 # 1024 * 2
n_mels = 40
normalized = False

sample_rate = 44100
power = 1.0

In [ ]:
sample_rate = 44100
power = 1.0

In [ ]:
mel_spectrogram = T.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=n_fft,
    # win_length=2*hop_length,
    hop_length=hop_length,
    f_min=0,
    f_max=sample_rate//2,
    n_mels=n_mels,
    power=power,
    window_fn=torch.hamming_window,
    normalized=normalized
)

In [ ]:
# path = './data/synthetic/train'

In [ ]:
# file = '/TUT-SED-synthetic-2016-mix-1.wav'

In [ ]:
#@title Folgend: Entweder files.upload nutzen oder drive mounten und dann den file path angeben

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Data_Code

/content/drive/.shortcut-targets-by-id/1nC9alf1XXss0MliDJ7UIwgKnGiUiWn01/Data_Code


In [ ]:
!ls

conda_dependencies.yml	experiments  models		   settings
data			LICENSE      pip_dependencies.txt  tools
data_feeders		main.py      README.md


In [ ]:
file = './data/2_5/audio/TUT-SED-synthetic-2016-mix-1.wav'

In [ ]:
# waveform, sample_rate = torchaudio.load(path + file)
waveform, sample_rate = torchaudio.load(file)

In [ ]:
waveform.shape, sample_rate

(torch.Size([1, 17667342]), 44100)

In [ ]:
mel = torch.log(mel_spectrogram(waveform))

In [ ]:
mel = mel.permute(0, 2, 1)

In [ ]:
mel.shape

torch.Size([1, 20032, 40])

In [ ]:
cnn_channels = 128
cnn_dropout = .25
rnn_in_dim = cnn_channels
rnn_out_dim = 128
nb_classes = 16

In [ ]:
from models import crnn

In [ ]:
crnn = crnn.CRNN(cnn_channels,
        cnn_dropout,
        rnn_in_dim,
        rnn_out_dim,
        nb_classes)

In [ ]:
# yhat = crnn(mel)

In [ ]:
# yhat.shape

In [ ]:
# yhat[0][:10, :]

In [ ]:
import pickle

In [ ]:
input_features = './data/mel_energies/features/TUT-SED-synthetic-2016-mix-1.cpickle'

In [ ]:
with open(input_features, 'rb') as f:
  mel = pickle.load(f, encoding='latin1')

In [ ]:
mel # : dict

{'feat': [array([[-10.34508092,  -9.63420605,  -9.3785564 , ...,  -7.38586424,
           -7.32913153,  -8.56749136],
         [-10.72728893,  -9.34900159,  -8.84533813, ...,  -8.05704906,
           -8.11169866,  -8.85618467],
         [-10.54756438,  -9.01082562,  -8.4896992 , ...,  -7.44385633,
           -7.42815323,  -8.27178612],
         ...,
         [-34.7840923 , -36.04365339, -36.04365339, ..., -36.04365339,
          -36.04365339, -36.04365339],
         [-34.7840923 , -36.04365339, -36.04365339, ..., -36.04365339,
          -36.04365339, -36.04365339],
         [-34.7840923 , -36.04365339, -36.04365339, ..., -36.04365339,
          -36.04365339, -36.04365339]])],
 'stat': [{'N': 20032,
   'S1': array([-115259.11952206, -102731.14337977, -101188.15576885,
          -102706.28861838, -101119.99394221,  -98423.85433891,
           -99572.34617272, -100621.21130485,  -99750.65733763,
          -100285.19488904, -101921.25185569, -101351.8296574 ,
          -102149.73888593, -1

In [ ]:
mel['feat'][0].shape

(20032, 40)

In [ ]:
num_frames = mel['feat'][0].shape[0]

In [ ]:
y = torch.randint(0, 2, (1, num_frames, nb_classes)).float()

In [ ]:
# apparently not working with GPU at the moment

# if torch.cuda.is_available():
#   device = torch.device("cuda")
# else:
#   device = torch.device("cpu")
device = 'cpu'

In [ ]:
mels = torch.Tensor(mel['feat'][0])

In [ ]:
mels = mels.to(device)

In [ ]:
mels = mels[None, :, :]

In [ ]:
mels.device

device(type='cpu')

In [ ]:
mels.shape, mels.dtype

(torch.Size([1, 20032, 40]), torch.float32)

In [ ]:
# 1st run, no training
yhat = crnn(mels)

In [ ]:
yhat

tensor([[[ 0.4776, -0.7150,  0.5877,  ..., -0.0171,  0.1392, -0.2506],
         [ 0.4710, -0.8152,  0.8156,  ...,  0.2151,  0.2985, -0.4244],
         [ 0.2393, -0.5635,  0.7400,  ..., -0.1045,  0.1204, -0.4307],
         ...,
         [ 0.3173, -0.0832,  1.1805,  ...,  0.1430,  0.8026, -0.6828],
         [ 0.6399, -0.3437,  1.5739,  ..., -0.0797,  0.8009, -1.3000],
         [ 0.6268, -0.0289,  1.2110,  ..., -0.0760,  0.5619, -0.7931]]],
       grad_fn=<CopySlices>)

In [ ]:
yhat.shape

torch.Size([1, 20032, 16])

In [ ]:
# loss

In [ ]:
objective = torch.nn.BCEWithLogitsLoss()

In [ ]:
objective(yhat, y)

tensor(0.6976, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [ ]:
# one sample trainig

In [ ]:
opt = torch.optim.Adam(crnn.parameters(), lr=1e-3)

In [ ]:
crnn.train()
loss = objective(yhat, y)
loss.backward()
opt.step()
opt.zero_grad()

In [ ]:
yhat2 = crnn(mels)

In [ ]:
yhat, yhat2

(tensor([[[ 0.0074,  0.1381, -0.0708,  ..., -0.3550,  0.2687, -0.2061],
          [-0.0169,  0.1518, -0.2288,  ..., -0.2907,  0.4380, -0.4196],
          [-0.2073,  0.2670, -0.4235,  ...,  0.0873,  0.4596, -0.3028],
          ...,
          [-0.2720,  0.6026, -0.0327,  ..., -0.3524,  0.8190,  0.1385],
          [ 0.1259,  0.7443, -0.7129,  ..., -0.5341,  1.1615, -0.0986],
          [-0.4370,  0.6316, -0.6828,  ..., -0.2948,  0.6574,  0.0221]]],
        grad_fn=<CopySlices>),
 tensor([[[-0.2373,  0.1826,  0.2778,  ...,  0.0718, -0.0807, -0.0360],
          [-0.5229,  0.1505,  0.0450,  ...,  0.0090,  0.1983, -0.0247],
          [-0.5722,  0.2954, -0.2117,  ...,  0.0376,  0.4354, -0.0147],
          ...,
          [-1.0547,  0.4844, -0.4353,  ..., -0.1546, -0.1553,  0.1130],
          [ 0.1983,  0.4897, -0.1870,  ...,  0.2150,  0.0329, -0.0563],
          [-0.0783,  0.5504, -0.2511,  ..., -0.4022, -0.0980, -0.1442]]],
        grad_fn=<CopySlices>))